# Assistant-01

### The OpenAI reference demo

https://platform.openai.com/docs/assistants/overview

### Get an OpenAI client

In [1]:
import common
import time

## Get an OpenAI client
client = common.get_openai_client()

print(client)

**Note:** The API version, the model version, and endpoint matter.

### Create an assistant

In [2]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo-test"
)
assistant_id = assistant.id

### Create an assistant thread

In [3]:
thread = client.beta.threads.create()
thread_id = thread.id
thread_id

'thread_4Hc1cWIDv2ntutiKGFqVmlZl'

### Create a user message in the assistant thread

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
message

ThreadMessage(id='msg_jDpHycRsSw5SvKfrnPqfie33', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1705035007, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_4Hc1cWIDv2ntutiKGFqVmlZl')

## Run the assistant in the assistant thread

In [5]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

### Wait until the run is completed

In [6]:
while True:
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
  if run.status == "completed":
    break
  time.sleep(1)

### Get the messages for the thread and print them

In [7]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
for msg in messages:
  #print(msg)
  content = msg.content
  for line in content:
    txt = line.text
    print(msg.role,txt.value)

assistant The solution to the equation \(3x + 11 = 14\) is \(x = 1\).
user I need to solve the equation `3x + 11 = 14`. Can you help me?


### List all running assistants in the model

In [8]:
# List the registered assistants
assistants = client.beta.assistants.list()
for assistant in assistants.data:
    print(assistant)


Assistant(id='asst_x0dByYj3yQJe3YVXzZumhHC9', created_at=1705035006, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-4-turbo-test', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])
Assistant(id='asst_dH8MRTuqwplfRBEDrnLJLjre', created_at=1705034667, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-4-turbo-test', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])
Assistant(id='asst_iZxyZmci2o3WqxE7oDSwALpF', created_at=1705034026, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-4-turbo-test', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])
Assistant(id='asst_9F6ZsUPuV

### Cleanup: clean the assistant and the thread thread

In [9]:
client.beta.assistants.delete(assistant_id=assistant_id)

client.beta.threads.delete(thread_id=thread_id)

ThreadDeleted(id='thread_4Hc1cWIDv2ntutiKGFqVmlZl', deleted=True, object='thread.deleted')